In [1]:
%%bash
set -e

cd /content

# Download micromamba into /content/bin/micromamba
curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba

# Create a clean env (isolated from Colab’s global packages)
./bin/micromamba create -y -n cb311 -c conda-forge python=3.11 pip

echo "✅ micromamba ready at /content/bin/micromamba"
echo "✅ env created: cb311"


bin/micromamba


Transaction

  Prefix: /root/.local/share/mamba/envs/cb311

  Updating specs:

   - python=3.11
   - pip


  Package                Version  Build                 Channel          Size
───────────────────────────────────────────────────────────────────────────────
  Install:
───────────────────────────────────────────────────────────────────────────────

  + _libgcc_mutex            0.1  conda_forge           conda-forge       3kB
  + _openmp_mutex            4.5  2_gnu                 conda-forge      24kB
  + bzip2                  1.0.8  hda65f42_8            conda-forge     260kB
  + ca-certificates   2025.11.12  hbd8a1cb_0            conda-forge     152kB
  + icu                     78.1  h33c6efd_0            conda-forge      13MB
  + ld_impl_linux-64        2.45  default_hbd61a6d_105  conda-forge     731kB
  + libexpat               2.7.3  hecca717_0            conda-forge      77kB
  + libffi                 3.5.2  h9ec8514_0            conda-forge      58kB
  

In [2]:
%%bash
set -euo pipefail

cd /content
MICROMAMBA="/content/bin/micromamba"

ts() { date +"[%Y-%m-%d %H:%M:%S]"; }

echo "$(ts) Sanity check micromamba:"
ls -lah "$MICROMAMBA"

echo "$(ts) Upgrading pip tooling inside cb311..."
"$MICROMAMBA" run -n cb311 python -m pip install -U pip setuptools wheel --progress-bar on

echo "$(ts) Installing PyTorch 2.5.1 (CUDA 12.1)... (this can take a while)"
"$MICROMAMBA" run -n cb311 pip install \
  --progress-bar on \
  torch==2.5.1+cu121 torchaudio==2.5.1+cu121 torchvision==0.20.1+cu121 \
  --index-url https://download.pytorch.org/whl/cu121

echo "$(ts) Installing ONNX (wheel)..."
"$MICROMAMBA" run -n cb311 pip install --progress-bar on onnx==1.16.0

echo "$(ts) Installing Chatterbox package (from GitHub, no-cache, upgrade)..."
"$MICROMAMBA" run -n cb311 pip uninstall -y chatterbox-tts chatterbox || true
"$MICROMAMBA" run -n cb311 pip install \
  --no-cache-dir --upgrade \
  --progress-bar on \
  "chatterbox-tts @ git+https://github.com/devnen/chatterbox-v2.git@master"

echo "$(ts) ✅ Installation complete!"


[2025-12-28 00:40:28] Sanity check micromamba:
-rwxrwxr-x 1 root root 17M Dec  4 14:39 /content/bin/micromamba
[2025-12-28 00:40:28] Upgrading pip tooling inside cb311...
[2025-12-28 00:40:29] Installing PyTorch 2.5.1 (CUDA 12.1)... (this can take a while)
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 37.4 MB/s  0:00:13
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 140.6 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 154.7 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 180.6 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 50.5 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 171.4 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 28.8 MB/s  0:00:13
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 24.1 MB/s  0:00:11
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/

  Running command git clone --filter=blob:none --quiet https://github.com/devnen/chatterbox-v2.git /tmp/pip-install-15e9duhr/chatterbox-tts_5da64d46bf68467cbd66d9a4db5cd7ee
  Running command git checkout -b master --track origin/master
  Switched to a new branch 'master'
  Branch 'master' set up to track remote branch 'master' from 'origin'.


In [3]:
%%bash
set -e

/content/bin/micromamba run -n cb311 python - <<'PY'
import inspect, torch
import chatterbox.tts_turbo as t

print("✅ torch:", torch.__version__)
print("✅ cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("✅ gpu:", torch.cuda.get_device_name(0))

print("✅ chatterbox.tts_turbo path:", t.__file__)

src = inspect.getsource(t.ChatterboxTurboTTS.from_pretrained)
print("\n--- from_pretrained() (first ~80 lines) ---")
print("\n".join(src.splitlines()[:80]))

# Heuristic check for the common buggy pattern that forces token=True semantics
markers = [" or True", "token=True", "token = True", "use_auth_token=True"]
hits = [m for m in markers if m in src]
print("\nHeuristic auth-forcing markers found:", hits)

if hits:
    raise SystemExit(
        "\n❌ This install still appears to force HF auth.\n"
        "Re-run Cell 2 (it already uses --no-cache-dir --upgrade).\n"
    )

print("\n✅ Looks good: Turbo should download without requiring user tokens.")
PY

✅ torch: 2.5.1+cu121
✅ cuda available: True
✅ gpu: Tesla T4
✅ chatterbox.tts_turbo path: /root/.local/share/mamba/envs/cb311/lib/python3.11/site-packages/chatterbox/tts_turbo.py

--- from_pretrained() (first ~80 lines) ---
    @classmethod
    def from_pretrained(cls, device) -> 'ChatterboxTurboTTS':
        # Check if MPS is available on macOS
        if device == "mps" and not torch.backends.mps.is_available():
            if not torch.backends.mps.is_built():
                print("MPS not available because the current PyTorch install was not built with MPS enabled.")
            else:
                print("MPS not available because the current MacOS version is not 12.3+ and/or you do not have an MPS-enabled device on this machine.")
            device = "cpu"

        local_path = snapshot_download(
            repo_id=REPO_ID,
            token=os.getenv("HF_TOKEN") or False,
            # Optional: Filter to download only what you need
            allow_patterns=["*.safetensors"

/root/.local/share/mamba/envs/cb311/lib/python3.11/site-packages/perth/perth_net/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [ ]:
# @title 4. Install Server + Run With Full Live Logs (foreground)
import os, time, subprocess, socket, requests
from pathlib import Path

PORT = 8004
REPO_DIR = "/content/Chatterbox-TTS-Server"
LOG_STDOUT = "/content/chatterbox_server_stdout.log"

def sh(cmd, check=False):
    return subprocess.run(["bash", "-lc", cmd], check=check)

def port_open(host="127.0.0.1", port=PORT, timeout=0.25):
    try:
        with socket.create_connection((host, port), timeout=timeout):
            return True
    except OSError:
        return False

os.chdir("/content")

# Fresh clone
sh("rm -rf /content/Chatterbox-TTS-Server", check=False)
sh("git clone https://github.com/devnen/Chatterbox-TTS-Server.git", check=True)
os.chdir(REPO_DIR)

print("=== Quick system checks ===")
sh("nvidia-smi || true", check=False)

print("\n=== Installing server requirements (prefer repo pins if present) ===")
if Path("requirements-nvidia.txt").exists():
    sh("/content/bin/micromamba run -n cb311 pip install -U pip setuptools wheel", check=False)
    sh("/content/bin/micromamba run -n cb311 pip install -r requirements-nvidia.txt", check=False)
else:
    sh(
        "/content/bin/micromamba run -n cb311 pip install -U pip setuptools wheel && "
        "/content/bin/micromamba run -n cb311 pip install "
        "fastapi 'uvicorn[standard]' pyyaml soundfile librosa safetensors "
        "python-multipart requests jinja2 watchdog aiofiles unidecode inflect tqdm "
        "pydub audiotsm praat-parselmouth",
        check=False
    )

print("\n=== Removing old stdout log ===")
Path(LOG_STDOUT).unlink(missing_ok=True)

print("\n=== Starting server with LIVE logs ===")
print("Log file:", LOG_STDOUT)
print("To stop the server, run Cell 5.\n")

env = os.environ.copy()
env["PYTHONUNBUFFERED"] = "1"

# Put HF cache somewhere inspectable/persistent for this runtime
env["HF_HOME"] = "/content/hf_home"
env["TRANSFORMERS_CACHE"] = "/content/hf_home/transformers"
env["HF_HUB_CACHE"] = "/content/hf_home/hub"
Path(env["HF_HOME"]).mkdir(parents=True, exist_ok=True)

proc = subprocess.Popen(
    ["/content/bin/micromamba", "run", "-n", "cb311", "python", "-u", "server.py"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
    env=env,
)

with open(LOG_STDOUT, "w", encoding="utf-8", errors="replace") as f:
    shown_link = False
    while True:
        line = proc.stdout.readline()
        if line:
            print(line, end="")
            f.write(line)
            f.flush()

        if (not shown_link) and port_open():
            shown_link = True
            print("\n=== Server port is reachable ===")
            print("Click the Colab proxy link below to open the Web UI.")
            from google.colab.output import serve_kernel_port_as_window
            serve_kernel_port_as_window(PORT)

            # Verify model load status via server endpoint
            try:
                mi = requests.get(f"http://127.0.0.1:{PORT}/api/model-info", timeout=2).json()
                print("\n/api/model-info:", mi)
            except Exception as e:
                print("\n/api/model-info query failed:", repr(e))

        if proc.poll() is not None:
            print("\n=== Server process exited with code", proc.returncode, "===")
            break

=== Quick system checks ===

=== Installing server requirements (prefer repo pins if present) ===

=== Removing old stdout log ===

=== Starting server with LIVE logs ===
Log file: /content/chatterbox_server_stdout.log
To stop the server, run Cell 5.

/root/.local/share/mamba/envs/cb311/lib/python3.11/site-packages/perth/perth_net/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename
/root/.local/share/mamba/envs/cb311/lib/python3.11/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Parselmouth library not found. Unvoiced segment removal feature will be disabled.
2025-12-28 01:02:47 [INFO] __main__: Starting T

<IPython.core.display.Javascript object>


/api/model-info: {'loaded': True, 'type': 'turbo', 'class_name': 'ChatterboxTurboTTS', 'device': 'cuda', 'sample_rate': 24000, 'supports_paralinguistic_tags': True, 'available_paralinguistic_tags': ['laugh', 'chuckle', 'sigh', 'gasp', 'cough', 'clear throat', 'sniff', 'groan', 'shush'], 'turbo_available_in_package': True}
INFO:     Uvicorn running on http://0.0.0.0:8004 (Press CTRL+C to quit)
2025-12-28 01:03:47 [INFO] server: Attempting to open web browser to: http://localhost:8004/
2025-12-28 01:05:50 [INFO] server: Request received for main UI page ('/').
2025-12-28 01:05:53 [INFO] server: Request received for /api/ui/initial-data.
2025-12-28 01:05:53 [INFO] utils: Found 28 predefined voices in /content/Chatterbox-TTS-Server/voices
2025-12-28 01:06:23 [INFO] server: Received /tts request: mode='predefined', format='wav'
2025-12-28 01:06:23 [INFO] server: Using predefined voice: Emily.wav
2025-12-28 01:06:23 [INFO] server: Splitting text into chunks of size ~240.
2025-12-28 01:06:23

In [ ]:
%%bash
PORT=8004

echo "PIDs listening on port $PORT:"
sudo lsof -t -i:$PORT || true

echo "Killing..."
sudo lsof -t -i:$PORT | xargs -r sudo kill -9

echo "Verify nothing is listening:"
sudo lsof -i:$PORT || true
